In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.head(1)

In [3]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [4]:
# age列が欠損値の行を除外、その他の変数の欠損値には文字列「欠損値」を補完
df1 = df.copy()
df1 = df1.astype({'class':str, 'deck':str})#category型のままだとfillnaでエラー
df1 = df1.dropna(subset=['age']).fillna('欠損値')
df1.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [5]:
xcol_list = ['survived', 'pclass', 'sex', 'sibsp', 'parch', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town','alive', 'alone']#'fare'は除外
ycol_list = ['age']

In [6]:
len(xcol_list)

13

In [7]:
import itertools
col_permutations_list = [cols for cols in itertools.permutations(xcol_list, 2)]
len(col_permutations_list)

156

In [9]:
import math
from scipy.special import comb
# 組み合わせ×階乗
comb(len(xcol_list), 2, exact=True)*math.factorial(2)

156

In [8]:
col_permutations_list[0:2]

[('survived', 'pclass'), ('survived', 'sex')]

In [11]:
def calc_median_and_rank(df, xcol1, xcol2, ycol):
    """
    対象(ycol)の中央値と中央値の順位を取得する
    
    Parameters
    ----------
    df：dataframe
    　対象のデータフレーム
    xcol1 : str
        変数1つ目
    xcol2 : str
        変数2つ目
    ycol : str
　　  中央値を取得する変数
    
    Returns
    -------
    df：dataframe
    　ycolの中央値とxcol1内におけるxcol2毎の中央値の順位を取得したデータフレーム
    """    
    df = df.groupby([xcol1, xcol2], as_index=False)[ycol].median().rename(columns={ycol: ycol + '_median'})
    
    df['Rank_Of_' + xcol2 + '_In_' + xcol1]=\
    df.groupby([xcol1], as_index=False)[ycol + '_median'].rank(method='min', ascending=True)
    
    # 確認用にソートしておく
    df = df.sort_values([xcol1,'Rank_Of_' + xcol2 + '_In_' + xcol1]).reset_index(drop=True)

    return df

In [12]:
# 関数の実行
df_median_and_rank_list =[]
for cols in col_permutations_list:
    out_df = calc_median_and_rank(df1, cols[0], cols[1], 'age')
    df_median_and_rank_list.append(out_df)

In [13]:
for df in df_median_and_rank_list[61:63]:
    display(df)

,embarked,pclass,age_median,Rank_Of_pclass_In_embarked
0,C,3,20.0,1.0
1,C,2,25.0,2.0
2,C,1,36.5,3.0
3,Q,3,21.5,1.0
4,Q,1,38.5,2.0
5,Q,2,43.5,3.0
6,S,3,25.0,1.0
7,S,2,30.0,2.0
8,S,1,37.0,3.0
9,欠損値,1,50.0,1.0


,embarked,sex,age_median,Rank_Of_sex_In_embarked
0,C,female,24.0,1.0
1,C,male,30.0,2.0
2,Q,female,21.5,1.0
3,Q,male,30.0,2.0
4,S,female,27.0,1.0
5,S,male,28.0,2.0
6,欠損値,female,50.0,1.0


In [14]:
def get_the_number_of_all_patterns(df, xcol1, xcol2, rank_col):
    """
    xcol2内におけるxcol1毎の順位のパターン毎の件数を取得
    
    Parameters
    ----------
    df：dataframe
    　対象のデータフレーム
    xcol1 : str
        変数1つ目
    xcol2 : str
        変数2つ目
    rank_col : str
　　  順位列のカラム
    
    Returns
    -------
    df_pivot_merge：dataframe
    　xcol2の情報が含まれたデータフレームと2変数全パターンの件数を取得したデータフレームを結合したデータフレーム(確認用として使用)
    df_summary : dataframe
      パターンの重複があるので、重複しないようxcol2を纏めたデータフレーム
    """        
    df_pivot = df.pivot_table(index=xcol1, columns=xcol2, values=rank_col, aggfunc='mean')
    # pivot_tableの列名を変更
    df_pivot.columns = [xcol2 + '_' + str(col) + '_rank' for col in df_pivot.columns.tolist()]
    # パターン件数取得用
    cnt_col = df_pivot.columns[df_pivot.columns.str.contains(xcol2)].tolist()
    # いずれかの列の順位が欠損値になっている行を削除
    df_pivot =df_pivot.reset_index().dropna()
    # 2変数全パターンの件数を取得したデータフレーム
    df_pattern = df_pivot.groupby(cnt_col, as_index=False).count().rename(columns={xcol1:'cnt'})
    # xcol2の情報が含まれたデータフレームと2変数全パターンの件数を取得したデータフレームを結合
    df_pivot_merge = pd.merge(df_pivot, df_pattern, on=cnt_col, how='left' )
    # パターンの重複があるので、重複しないようgroupbyでxcol2を1つのリストに纏めたデータフレーム
    #df_summary = df_pivot_merge.groupby(cnt_col+['cnt'], as_index=False)[xcol1].apply(','.join).reset_index().rename(columns={0:xcol1})
    df_summary = df_pivot_merge.groupby(cnt_col+['cnt'], as_index=False)[xcol1].apply(list).reset_index().rename(columns={0:xcol1})
    
    return df_pivot_merge, df_summary

In [15]:
# 関数の実行
df_check_list = []
df_get_the_number_of_all_patterns_list =[]
# 2つのリストはどちらも同数格納されている
for cols, df in zip(col_permutations_list, df_median_and_rank_list):
    rank_col_name = df.columns[df.columns.str.contains('Rank')][0]
    out_df1, out_df2 = get_the_number_of_all_patterns(df, cols[0], cols[1], rank_col_name)
    df_check_list.append(out_df1)
    df_get_the_number_of_all_patterns_list.append(out_df2)

In [16]:
for df in df_get_the_number_of_all_patterns_list[61:63]:
    display(df)

,pclass_1_rank,pclass_2_rank,pclass_3_rank,cnt,embarked
0,2.0,3.0,1.0,1,[Q]
1,3.0,2.0,1.0,2,"[C, S]"


,sex_female_rank,sex_male_rank,cnt,embarked
0,1.0,2.0,3,"[C, Q, S]"


In [17]:
# 確認用
for df in df_check_list[61:63]:
    display(df)

,embarked,pclass_1_rank,pclass_2_rank,pclass_3_rank,cnt
0,C,3.0,2.0,1.0,2
1,Q,2.0,3.0,1.0,1
2,S,3.0,2.0,1.0,2


,embarked,sex_female_rank,sex_male_rank,cnt
0,C,1.0,2.0,3
1,Q,1.0,2.0,3
2,S,1.0,2.0,3


In [18]:
import math

def calc_probability(df):
    """
    順位のパターンの確率を算出する
    
    Parameters
    ----------
    df：dataframe
    　対象のデータフレーム
   
    Returns
    -------
    df：dataframe
    　順位のパターンの確率を追加したデータフレーム

    """     
    # 全事象を取得
    all_events = df['cnt'].sum()
    df['all_events'] = all_events
    # パターン件数を全事象で除算した値を算出
    df['probability'] = df.apply(lambda x: x['cnt'] / x['all_events'] if x['all_events']!=0 else 0, axis=1)
    
    return df

In [19]:
# 関数の実行
df_get_probability_list =[]
# 2つのリストはどちらも同数格納されている
for df in df_get_the_number_of_all_patterns_list:
    # データフレームが空の場合はエラーが発生する
    # (ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series)ため条件分岐
    if df.shape[0]==0:
        pass
    else:
        out_df = calc_probability(df)
        df_get_probability_list.append(out_df)

In [20]:
for df in df_get_probability_list[61:63]:
    display(df)

,class_First_rank,class_Second_rank,class_Third_rank,cnt,embarked,all_events,probability
0,2.0,3.0,1.0,1,[Q],3,0.333333
1,3.0,2.0,1.0,2,"[C, S]",3,0.666667


,who_child_rank,who_man_rank,who_woman_rank,cnt,embarked,all_events,probability
0,1.0,2.0,2.0,2,"[C, S]",3,0.666667
1,1.0,3.0,2.0,1,[Q],3,0.333333


In [21]:
# https://www.haya-programming.com/entry/2018/01/22/151849
from math import log2

def calc_entropy(prob_list):
    """
    順位のパターンの平均情報量を算出する
    
    Parameters
    ----------
    prob_list：list
    　確率の値が含まれているリスト
   
    Returns
    -------
    平均情報量の値

    """  
    return -sum([x*log2(x) if x != 0 else 0 for x in prob_list])

In [23]:
df_H_list = []

for df in df_get_probability_list:
    df['entropy'] = calc_entropy(df['probability'].tolist())
    df_H_list.append(df)

In [24]:
for df in df_H_list[61:63]:
    display(df)

,class_First_rank,class_Second_rank,class_Third_rank,cnt,embarked,all_events,probability,entropy
0,2.0,3.0,1.0,1,[Q],3,0.333333,0.918296
1,3.0,2.0,1.0,2,"[C, S]",3,0.666667,0.918296


,who_child_rank,who_man_rank,who_woman_rank,cnt,embarked,all_events,probability,entropy
0,1.0,2.0,2.0,2,"[C, S]",3,0.666667,0.918296
1,1.0,3.0,2.0,1,[Q],3,0.333333,0.918296


In [25]:
H_list = []
for cols, df in zip(col_permutations_list, df_get_probability_list):
    H_list.append({'factor':cols[0], 'level':cols[1], 'entropy':calc_entropy(df['probability'].tolist())})

In [26]:
H_list

[{'factor': 'survived', 'level': 'pclass', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'sex', 'entropy': 1.0},
 {'factor': 'survived', 'level': 'sibsp', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'parch', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'embarked', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'class', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'who', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'adult_male', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'deck', 'entropy': 1.0},
 {'factor': 'survived', 'level': 'embark_town', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'alive', 'entropy': -0.0},
 {'factor': 'survived', 'level': 'alone', 'entropy': -0.0},
 {'factor': 'pclass', 'level': 'survived', 'entropy': -0.0},
 {'factor': 'pclass', 'level': 'sex', 'entropy': -0.0},
 {'factor': 'pclass', 'level': 'sibsp', 'entropy': -0.0},
 {'factor': 'pclass', 'level': 'parch', 'entropy': -0.0},
 {'factor': 'pclass', 'level': 'embarke

In [27]:
gc.collect()

1487